# Install

In [ ]:
!pip install langchain faiss-cpu langchain_openai ragas langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade langchain faiss-cpu

# API

In [ ]:
API_ENDPOINT = "https://ai-research-proxy.azurewebsites.net"
API_KEY = "AAA"

# Embedding (only use if no FAISS database yet)

In [ ]:
# Imports
import os
import json
import shutil
from google.colab import files
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
# Embedding model
embedding = OpenAIEmbeddings(
    openai_api_key=API_KEY,
    model="text-embedding-ada-002"
)

def get_directory_size(path):
    total = 0
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            total += os.path.getsize(os.path.join(dirpath, f))
    return total

def embed_in_batches(docs, embedding, batch_size=100):
    """
    Embed documents in batches and return a combined FAISS store.
    """
    vectorstore = None
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i+batch_size]
        print(f"  → Embedding batch {i} to {i+len(batch)}...")
        batch_store = FAISS.from_documents(batch, embedding)

        if vectorstore is None:
            vectorstore = batch_store
        else:
            vectorstore.merge_from(batch_store)
    return vectorstore

# Upload and process file
print("Upload your JSON files:")
uploaded = files.upload()

vectorstore = None

for filename in uploaded.keys():
    print(f"\nProcessing: {filename}")

    with open(filename, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
        except Exception as e:
            print(f"Failed to parse {filename}: {e}")
            continue

    docs = []

    for item in data.get("chunks", []):
        chunk_id = item.get("chunk_id")
        chunk_type = item.get("chunk_type", "")
        source_file = item.get("source_file", filename)
        page_num = item.get("page_num", None)

        metadata = {
            "chunk_id": chunk_id,
            "source_file": source_file,
            "page_num": page_num
        }

        if chunk_type == "text":
            text = item.get("text", "")
            if text.strip():
                docs.append(Document(
                    page_content=text,
                    metadata=metadata
                ))

        elif chunk_type == "table":
            table = item.get("table", {})
            table_str = "\n".join([
                " | ".join(row.values())
                for row in table.values()
            ])
            docs.append(Document(
                page_content=table_str,
                metadata=metadata
            ))

    if not docs:
        print(f"No documents extracted from {filename}")
        continue

    print(f"Embedding {len(docs)} segments in batches...")
    temp_store = embed_in_batches(docs, embedding, batch_size=100)

    if vectorstore is None:
        vectorstore = temp_store
    else:
        vectorstore.merge_from(temp_store)

    vectorstore.save_local("faiss_index")
    print(f"Stored {filename} → FAISS index updated.")

# Zip and download
size_mb = get_directory_size("faiss_index") / (1024 * 1024)
print(f"\nFinal FAISS index size: {size_mb:.2f} MB")

shutil.make_archive("faiss_index", 'zip', "faiss_index")
os.rename("faiss_index.zip", "faiss_27reports.zip")
files.download("faiss_27reports.zip")

Upload your JSON files:


Saving Aer Lingus Sustainability_chunked.json to Aer Lingus Sustainability_chunked.json
Saving Air Iberia Sustainability_chunked.json to Air Iberia Sustainability_chunked.json
Saving Air NZ GHG_chunked.json to Air NZ GHG_chunked.json
Saving Air NZ Sustainability_chunked.json to Air NZ Sustainability_chunked.json
Saving Alaska airlines Sustainability_chunked.json to Alaska airlines Sustainability_chunked.json
Saving American Airlines Sustainaibility_chunked.json to American Airlines Sustainaibility_chunked.json
Saving ANA annual report _chunked.json to ANA annual report _chunked.json
Saving Cathay Pacific Sustainability_chunked.json to Cathay Pacific Sustainability_chunked.json
Saving Delta Sustainability_chunked.json to Delta Sustainability_chunked.json
Saving DHL Sustainability_chunked.json to DHL Sustainability_chunked.json
Saving Emirates Report_chunked.json to Emirates Report_chunked.json
Saving FedEX sustainability_chunked.json to FedEX sustainability_chunked.json
Saving finnair a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Access LLM & test

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(
    openai_api_base= API_ENDPOINT,
    openai_api_key= API_KEY,
    model = "gpt-4o-mini",
    temperature=0.1,
)

messages = [
    SystemMessage(
        content="You are a helpful assistant that im using to make a test request to."
    ),
    HumanMessage(
        content="test from litellm. tell me why it's amazing in 1 sentence"
    ),
]
response = chat(messages)

print(response)

/tmp/ipython-input-4-2371223853.py:20: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat(messages)


content='Litellm is amazing because it combines advanced language processing capabilities with user-friendly interfaces, making it accessible and powerful for a wide range of applications.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 41, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-Bk7TXsSJOmeecs7slw5DYCg5vfgxb', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--b4c34e3b-47d6-4c5f-b398-fa104d49bf44-0' usage_metadata={'input_tokens': 41, 'output_tokens': 30, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


From https://ai-research-proxy.azurewebsites.net/ui/?userID=e8fc58ee-ae1a-4026-9a80-8fa56df2b6bd&page=api_ref

# Unzip

In [ ]:
# Upload .zip
from google.colab import files
uploaded = files.upload()

Saving faiss_27reports.zip to faiss_27reports.zip


In [ ]:
import zipfile

with zipfile.ZipFile("faiss_27reports.zip", "r") as zip_ref:
    zip_ref.extractall("faiss_index")

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(
    openai_api_base=API_ENDPOINT,
    openai_api_key=API_KEY,
    model="text-embedding-ada-002"
)

vectorstore = FAISS.load_local("faiss_index", embedding, allow_dangerous_deserialization=True)
print("FAISS index loaded!")


/tmp/ipython-input-7-295396684.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(


FAISS index loaded!


# Imports and Retriever

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from operator import itemgetter
import re

from IPython.display import display, Markdown

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Question

In [ ]:
question = "How much SAF did KLM use in 2024?"

# Naive

Straight up takes the question and finds the most relevant parts of the documents to answer it.

Usually not as good as advanced techniques, especially for more complex questions.

In [ ]:
doc_references = "\n\n".join(
    f"Title: {doc.metadata.get('source_file', 'Unknown Source')}\nPage: {doc.metadata.get('page_num', 'Unknown Page')}\nContent: {doc.page_content}"
    for doc in retriever.invoke(question)
)


naive_template = [
    SystemMessage(content=""" You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.
You reference the sources in a reference list at the end, using the original source file name under 'source_file' and the page number under 'page_num'. In the list, don't repeat the same reference multiple times.
You can recognize when no document is a good match and return "I don't know". """),
    HumanMessage(content="Here are the documents: " + doc_references),
    HumanMessage(content="Here are the instructions: " + question)
]

display(Markdown(chat.invoke(naive_template).content))


In 2024, KLM used 57 kilotons of Sustainable Aviation Fuel (SAF). 

References:
- KLM Sustainability, page 42.

# Advanced

All codes adapted from:
https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

## Decomposition - Subquestions

It creates subquestion of the question asked by the user.

**Recursively**:

Answers each sub-question in sequence, carrying forward the context from previous ones.


**Individually**:

Answers each subquestion individually, then synthesize them in a single answer.

In [ ]:
subquestions_template = """You are a helpful assistant that generates multiple sub-questions related to an input question, the question regards the one or more aviation sustainability report.
The goal is to break down the input into a set of sub-questions that can be answered in isolation.
Generate multiple search queries related to: {question}
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(subquestions_template)

In [ ]:
# Chain
generate_queries_decomposition = ( prompt_decomposition | chat | StrOutputParser() | (lambda x: x.split("\n")))

# Run
subquestions = generate_queries_decomposition.invoke({"question":question})

In [ ]:
subquestions

['1. What is the total amount of Sustainable Aviation Fuel (SAF) used by KLM in 2024?',
 "2. How does KLM's SAF usage in 2024 compare to previous years?",
 "3. What percentage of KLM's total fuel consumption in 2024 was made up of SAF?"]

## Decomposition - Recursively answered

In [ ]:
# RECURSIVE
recursive_template = """You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.

Here is the question you need to answer fully:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question (each chunk may include source metadata):

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question fully.

When answering, clearly state the source and page number in parentheses after each key fact or figure.
If information is drawn from the context, include a citation in this format: (source: {{source_file}}, page {{page_num}}).

Answer with references:
Add a reference list at the end, listing all the references used in format: source: {{source_file}}, page {{page_num}}. In the list, don't repeat the same reference multiple times.
"""


recursive_prompt = ChatPromptTemplate.from_template(recursive_template)

In [ ]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()

q_a_pairs = ""
for q in subquestions:

    rag_chain = (
    {"context": itemgetter("question") | retriever,
     "question": itemgetter("question"),
     "q_a_pairs": itemgetter("q_a_pairs")}
    | recursive_prompt
    | chat
    | StrOutputParser())

    recursive_answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,recursive_answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

display(Markdown(recursive_answer))


In 2024, KLM's total fuel consumption was 3,273 kilotons of conventional aviation fuel, and it used 57 kilotons of Sustainable Aviation Fuel (SAF) (source: KLM Sustainability, page 42). To calculate the percentage of KLM's total fuel consumption that was made up of SAF, we can use the following formula:

\[
\text{Percentage of SAF} = \left( \frac{\text{SAF used}}{\text{Total fuel consumption}} \right) \times 100
\]

Substituting the values:

\[
\text{Percentage of SAF} = \left( \frac{57 \text{ kilotons}}{3,273 \text{ kilotons}} \right) \times 100 \approx 1.74\%
\]

Thus, approximately **1.74%** of KLM's total fuel consumption in 2024 was made up of Sustainable Aviation Fuel (SAF).

### References
source: KLM Sustainability, page 42

## Decomposition - Answered individually

In [ ]:
# INDIVIDUAL
individual_template0 = ChatPromptTemplate.from_template(
    """
You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.

Answer the question based on the context below.

Context:
{context}

Question:
{question}

When answering, clearly state the source and page number in parentheses after each key fact or figure.

If information is drawn from the context, include a citation in this format: (source: {{source_file}}, page {{page_num}}).

Answer with references:


Add a reference list at the end, listing all the references used in format: source: {{source_file}}, page {{page_num}}. In the list, don't repeat the same reference multiple times.
"""
)

def individual(question, individual_template, sub_question_generator_chain):
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    individual_results = []
    all_docs = []
    for sub_question in sub_questions:
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        all_docs.extend(retrieved_docs)


        context_text = ""
        for doc in retrieved_docs:
            source_info = f"(Source: {doc.metadata.get('source_file', 'Unknown')}, Page: {doc.metadata.get('page_num', 'N/A')})"
            context_text += f"{doc.page_content}\n{source_info}\n\n"

        answer_individual0 = (individual_template | chat | StrOutputParser()).invoke({
            "context": context_text,
            "question": sub_question
        })
        individual_results.append(answer_individual0)

    return individual_results, sub_questions, all_docs ##

answers_i, subquestions_i, all_docs_i = individual(question, individual_template0, generate_queries_decomposition)

/tmp/ipython-input-17-1419450673.py:33: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [ ]:
def format_qa_pairs(subquestions, answers):
    """Format Q and A pairs"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(subquestions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(subquestions_i, answers_i)

# Prompt
individual_template1 = """
You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.

Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}

When answering, clearly state the source and page number in parentheses after each key fact or figure.

If information is drawn from the context, include a citation in this format: (source: {{source_file}}, page {{page_num}}).

Answer with references:

Add a reference list at the end, listing all the references used in format: source: {{source_file}}, page {{page_num}}. In the list, do not repeat the same reference more than one time.
"""

individual_prompt = ChatPromptTemplate.from_template(individual_template1)

individual_final = (
    individual_prompt
    | chat
    | StrOutputParser()
)

individual_answer= individual_final.invoke({"context":context,"question":question})

display(Markdown(individual_answer))

In 2024, KLM used a total of 57 kilotons (or 57,000 tonnes) of Sustainable Aviation Fuel (SAF) (source: KLM Sustainability, page 42). This represents a significant increase from the previous year, where KLM consumed 49 kilotons of SAF in 2023 (source: KLM Sustainability, page 42).

### References:
source: KLM Sustainability, page 42.

## RAG-Fusion

It rephrases the question a defined number of times, keeping its meaning.

Example: "How much SAF did KLM use in 2024?" → "What is KLM SAF usage in 2024?", ...

In [ ]:
RAGf_template = """
The user will submit some documents containing the 2023-2024 reports of some aviation companies.
You follow these instructions only.
You are a helpful assistant that generates multiple search queries based on a single input query.
Generate multiple search queries related to: {question}
Output (3 queries):"""

RAGf_prompt = ChatPromptTemplate.from_template(RAGf_template)

generate_queries = (
    RAGf_prompt
    | chat
    | StrOutputParser()
    | (lambda x: x.strip().split("\n")[:3])
)

def reciprocal_rank_fusion(results: list[list], k=60):
    """Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)


    reranked_results = [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    return [doc for doc, _ in reranked_results]


retrieval_RAGf = generate_queries | retriever.map() | reciprocal_rank_fusion

docs = retrieval_RAGf.invoke({question})

formatted_docs = "\n\n".join(
    f"Source: {doc.metadata.get('source_file', 'Unknown Source')}\n"
    f"Page: {doc.metadata.get('page_num', 'Unknown Page')}\n"
    f"Content: {doc.page_content}"
    for doc in docs
)


RAGf_template = """
You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.

Answer the following question using only the provided context.

{context}

Question: {question}

When quoting a fact, cite it like this:
(source: {{source_file}}, page {{page_num}})

At the very end add a reference list, one per line, e.g.:
- source: {{source_file}}, page {{page_num}}. In the list, do not repeat the same reference more than one time.

Answer:
"""

RAGf_prompt = ChatPromptTemplate.from_template(RAGf_template)

prepare_input = RunnablePassthrough.assign(
    context=itemgetter("context"),
    question=itemgetter("question")
)

RAGf_final = (
    prepare_input
    | RAGf_prompt
    | chat
    | StrOutputParser()
)

RAGf_answer= RAGf_final.invoke({"question":question, "context": formatted_docs})

from IPython.display import display, Markdown
display(Markdown(RAGf_answer))

/tmp/ipython-input-19-1257401244.py:33: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)


In 2024, KLM used 57 Ktons of Sustainable Aviation Fuel (SAF), which represents a 17.1% increase from the 49 Ktons used in 2023 (source: KLM Sustainability, page 42).

Reference list:
- source: KLM Sustainability, page 42.

## HyDE

First generates an hypothetical answer, and then uses it instead of the question to retrieve the relevant information.

The concept is that the question should be clarified and rewritten in terms used in the documents.

In [ ]:
# HyDE document genration (no use of retrieved docs, hypothetical answer)
HyDE_template0 = """Please answer the question
Question: {question}
Passage:"""
HyDE_prompt0 = ChatPromptTemplate.from_template(HyDE_template0)


HyDE_retrieval0 = (
    HyDE_prompt0 | chat | StrOutputParser()
)

hypothetical_ans = HyDE_retrieval0.invoke({"question": question})
display(Markdown(hypothetical_ans))

# Use that hypothetical ans for retrieval
HyDE_retrieval = retriever.invoke(hypothetical_ans)

I'm sorry, but I don't have access to specific data or events that occurred in 2024, including the amount of Sustainable Aviation Fuel (SAF) used by KLM. You may want to check KLM's official reports or news releases for the most accurate and up-to-date information.

In [ ]:
# RAG
HyDE_template = """
You are a document analyst.
You follow these instructions and use only the documents provided.

Answer the following question based on this context:

{context}

Question: {question}

Instructions:
- When answering, clearly state the source and page number in parentheses after each key fact or figure.
- Use this exact format for citations: (source: {{source_file}}, page {{page_num}})
- If multiple facts come from the same page, cite it each time in the text, but only list it ONCE in the reference list.
- Do NOT repeat the same source and page combination more than once in the reference list.

Answer with references:

At the end, add a 'Reference List' with only UNIQUE entries in the format:
- source: {{source_file}}, page {{page_num}}
"""

HyDE_prompt = ChatPromptTemplate.from_template(HyDE_template)

HyDE_final = (
    HyDE_prompt
    | chat
    | StrOutputParser()
)

HyDE_answer= HyDE_final.invoke({"context":HyDE_retrieval,"question":question})

display(Markdown(HyDE_answer))

In 2024, KLM used a total of 57 kilotons of Sustainable Aviation Fuel (SAF), which represents a 17.1% increase from the previous year (source: KLM Sustainability, page 42). Additionally, the SAF savings achieved by KLM amounted to 165 kilotons (source: KLM Sustainability, page 42).

Reference List:
- source: KLM Sustainability, page 42

# ALL Q&A

In [ ]:
print(f"question: {question}\n\n")


print("Naive answer:")
display(Markdown(chat.invoke(naive_template).content))

print("\n\nRecursive answer:")
display(Markdown(recursive_answer))

print("\n\nIndividual answer:")
display(Markdown(individual_answer))

print("\n\nRAG-Fusion answer:")
display(Markdown(RAGf_answer))

print("\n\nHyDE answer:")
display(Markdown(HyDE_answer))


question: How much SAF did KLM use in 2024?


Naive answer:


In 2024, KLM used 57 kilotons of Sustainable Aviation Fuel (SAF). 

References:
- KLM Sustainability, page 42.



Recursive answer:


In 2024, KLM's total fuel consumption was 3,273 kilotons of conventional aviation fuel, and it used 57 kilotons of Sustainable Aviation Fuel (SAF) (source: KLM Sustainability, page 42). To calculate the percentage of KLM's total fuel consumption that was made up of SAF, we can use the following formula:

\[
\text{Percentage of SAF} = \left( \frac{\text{SAF used}}{\text{Total fuel consumption}} \right) \times 100
\]

Substituting the values:

\[
\text{Percentage of SAF} = \left( \frac{57 \text{ kilotons}}{3,273 \text{ kilotons}} \right) \times 100 \approx 1.74\%
\]

Thus, approximately **1.74%** of KLM's total fuel consumption in 2024 was made up of Sustainable Aviation Fuel (SAF).

### References
source: KLM Sustainability, page 42



Individual answer:


In 2024, KLM used a total of 57 kilotons (or 57,000 tonnes) of Sustainable Aviation Fuel (SAF) (source: KLM Sustainability, page 42). This represents a significant increase from the previous year, where KLM consumed 49 kilotons of SAF in 2023 (source: KLM Sustainability, page 42).

### References:
source: KLM Sustainability, page 42.



RAG-Fusion answer:


In 2024, KLM used 57 Ktons of Sustainable Aviation Fuel (SAF), which represents a 17.1% increase from the 49 Ktons used in 2023 (source: KLM Sustainability, page 42).

Reference list:
- source: KLM Sustainability, page 42.



HyDE answer:


In 2024, KLM used a total of 57 kilotons of Sustainable Aviation Fuel (SAF), which represents a 17.1% increase from the previous year (source: KLM Sustainability, page 42). Additionally, the SAF savings achieved by KLM amounted to 165 kilotons (source: KLM Sustainability, page 42).

Reference List:
- source: KLM Sustainability, page 42

# Evaluation: Ragas

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

## Questions and Ground Truths

In [ ]:
import json
import os
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness, faithfulness, NonLLMContextRecall
from ragas.llms import LangchainLLMWrapper

# Upload and load JSON file
from google.colab import files
uploaded = files.upload()

file_name = next(iter(uploaded))
with open(file_name, "r") as f:
    ground_truth_data = json.load(f)

# Create GROUND_TRUTH_MAP from file
GROUND_TRUTH_MAP = {
    item["question"]: {
        "ground_truth": item["ground_truth_answer"]
    }
    for item in ground_truth_data
}


Saving Evaluation.json to Evaluation.json


## Models

In [ ]:
#NAIVE

def run_naive(question: str) -> dict:
    docs = retriever.invoke(question)

    doc_references = "\n\n".join(
        f"Title: {doc.metadata.get('source_file', 'Unknown Source')}\nPage: {doc.metadata.get('page_num', 'Unknown Page')}\nContent: {doc.page_content}"
        for doc in docs
    )

    messages = [
        SystemMessage(content=""" You are a document analyst.
The user will submit some documents containing the 2023-2024 reports of some aviation companies, the name of the company is under 'source file'.
You follow these instructions and use only these documents.
You reference the sources in a reference list at the end, using the original source file name under 'source_file' and the page number under 'page_num'. In the list, don't repeat the same reference multiple times.
You can recognize when no document is a good match and return "I don't know". """),
        HumanMessage(content="Here are the documents: " + doc_references),
        HumanMessage(content="Here are the instructions: " + question)
    ]

    answer = chat(messages).content

    ground_truth_info = GROUND_TRUTH_MAP.get(question, {})
    ground_truth_answer = ground_truth_info.get("ground_truth", [])

    return {
        "model_id": "naive",
        "question": question,
        "answer": answer,
        "contexts": [doc.page_content for doc in docs],
        "ground_truth": ground_truth_answer
    }


In [ ]:
# Recursive
def run_recursive(question: str) -> dict:
    # Subquestions
    subquestions = generate_queries_decomposition.invoke({"question": question})

    q_a_pairs = ""
    all_docs = []

    for subq in subquestions:
        # Retrieve docs for each subquestion
        docs = retriever.invoke(subq)
        all_docs.extend(docs)

        context_text = "\n\n".join(doc.page_content for doc in docs)

        # Format prompt for subquestion
        formatted_prompt = recursive_prompt.format_prompt(
            question=subq,
            context=context_text,
            q_a_pairs=q_a_pairs or "None yet"
        )

        # Invoke chat with formatted prompt
        sub_answer = chat.invoke(formatted_prompt).content

        # Append Q&A pair
        q_a_pairs += "\n---\n" + format_qa_pair(subq, sub_answer)

    # Aggregate all docs content for final prompt context
    final_context = "\n\n".join(doc.page_content for doc in all_docs)

    # Format final prompt for the original question
    final_prompt = recursive_prompt.format_prompt(
        question=question,
        context=final_context,
        q_a_pairs=q_a_pairs or "None"
    )

    # Final answer
    final_answer = chat.invoke(final_prompt).content

    # Deduplicate contexts
    unique_contexts = list({doc.page_content for doc in all_docs})

    # Use GROUND_TRUTH_MAP to get the ground truth answer
    ground_truth_info = GROUND_TRUTH_MAP.get(question, {})
    ground_truth_answer = ground_truth_info.get("ground_truth", [])

    return {
        "model_id": "recursive",
        "question": question,
        "answer": final_answer,
        "contexts": unique_contexts,
        "ground_truth": ground_truth_answer
    }


In [ ]:
# Individual
def run_individual(question: str) -> dict:
    sub_questions = generate_queries_decomposition.invoke({"question": question})

    subanswers = []
    all_docs = []

    for subq in sub_questions:
        docs = retriever.invoke(subq)
        all_docs.extend(docs)

        # Context string
        context = "\n\n".join(
            f"{doc.page_content}\n(Source: {doc.metadata.get('source_file', 'Unknown')}, Page: {doc.metadata.get('page_num', 'N/A')})"
            for doc in docs
        )

        # Format the individual_template0 prompt with context and question
        formatted_prompt = individual_template0.format_prompt(
            context=context,
            question=subq
        )

        # Invoke chat and get answer content
        answer = chat.invoke(formatted_prompt).content

        subanswers.append((subq, answer))

    # Format Q+A pairs text for final synthesis prompt
    q_a_pairs_text = "\n\n".join(
        f"Question: {q}\nAnswer: {a}" for q, a in subanswers
    )

    # Format final prompt to synthesize answers
    final_prompt = individual_prompt.format_prompt(
        context=q_a_pairs_text,
        question=question
    )


    final_answer = chat.invoke(final_prompt).content

    # Deduplicate contexts
    unique_contexts = list({doc.page_content for doc in all_docs})

    # Use GROUND_TRUTH_MAP to get the ground truth answer
    ground_truth_info = GROUND_TRUTH_MAP.get(question, {})
    ground_truth_answer = ground_truth_info.get("ground_truth", [])

    return {
        "model_id": "individual",
        "question": question,
        "answer": final_answer,
        "contexts": unique_contexts,
        "ground_truth": ground_truth_answer
    }


In [ ]:
# RAG-Fusion
def run_ragf(question: str) -> dict:
    docs = retrieval_RAGf.invoke({"question": question})

    context = "\n\n".join(doc.page_content for doc in docs)

    answer = RAGf_final.invoke({
        "question": question,
        "context": context
    })

    # Use GROUND_TRUTH_MAP to get the ground truth answer
    ground_truth_info = GROUND_TRUTH_MAP.get(question, {})
    ground_truth_answer = ground_truth_info.get("ground_truth", [])

    return {
        "model_id": "rag_fusion",
        "question": question,
        "answer": answer,
        "contexts": [doc.page_content for doc in docs],
        "ground_truth": ground_truth_answer
    }



# HyDE
def run_hyde(question: str) -> dict:
    hypothetical_answer = HyDE_retrieval0.invoke({"question": question})
    docs = retriever.invoke(hypothetical_answer)

    context = "\n\n".join(doc.page_content for doc in docs)

    answer = HyDE_final.invoke({
        "context": context,
        "question": question
    })

    # Use GROUND_TRUTH_MAP to get the ground truth answer
    ground_truth_info = GROUND_TRUTH_MAP.get(question, {})
    ground_truth_answer = ground_truth_info.get("ground_truth", [])

    return {
        "model_id": "hyde",
        "question": question,
        "answer": answer,
        "contexts": [doc.page_content for doc in docs],
        "ground_truth": ground_truth_answer
    }


## Evaluation

In [ ]:
from ragas.metrics import AnswerCorrectness, faithfulness, LLMContextRecall
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from datasets import Dataset
import os
import pandas as pd
import time
from collections import defaultdict

# Environment
os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = API_ENDPOINT

evaluator_llm  = LangchainLLMWrapper(chat)
context_recall = LLMContextRecall(llm=evaluator_llm)

answer_correctness = AnswerCorrectness(weights=[0.75, 0.25])

all_results = []
model_times = defaultdict(float)

questions_to_run = list(GROUND_TRUTH_MAP.keys())

for question in questions_to_run:
    for run_fn in [run_naive, run_recursive, run_individual, run_ragf, run_hyde]:
        model_id = run_fn.__name__.replace("run_", "")
        start = time.time()
        result = run_fn(question)
        end = time.time()
        model_times[model_id] += end - start
        all_results.append(result)

# Print total time per model
print("Total time per model:")
for model_id, total_time in model_times.items():
    print(f"  {model_id}: {total_time:.2f} seconds")


dataset = Dataset.from_list(all_results)

# Evaluate
ragas_scores = evaluate(
    dataset=dataset,
    metrics=[answer_correctness, faithfulness, context_recall]
)

# Convert scores to pandas
df_scores = ragas_scores.to_pandas()

# Convert original results to pandas
df_results = pd.DataFrame(all_results)

# Merge on question + answer
df = pd.concat([df_results[['model_id', 'question']], df_scores], axis=1)

# Group by model_id
overall_scores = (
    df.groupby("model_id")[["answer_correctness", "context_recall", "faithfulness"]]
      .mean()
      .round(3)
      .sort_values("answer_correctness", ascending=False)
)

print(overall_scores)

Total time per model:
  naive: 45.04 seconds
  recursive: 466.59 seconds
  individual: 326.20 seconds
  ragf: 76.67 seconds
  hyde: 101.01 seconds


Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

            answer_correctness  context_recall  faithfulness
model_id                                                    
individual               0.620           0.702         0.833
rag_fusion               0.586           0.636         0.751
hyde                     0.563           0.474         0.922
naive                    0.540           0.512         0.617
recursive                0.481           0.711         0.859


In [ ]:
# CHECK! should save a nice structured json. instead of previous so you can save the different runs (5?)

import json
from collections import defaultdict


def remove_contexts(d):
    if isinstance(d, dict):
        return {k: remove_contexts(v) for k, v in d.items() if k not in ['context', 'contexts']}
    elif isinstance(d, list):
        return [remove_contexts(i) for i in d]
    else:
        return d

structured_results = defaultdict(dict)

for res in all_results:
    question = res['question']
    model_id = res['model_id']
    answer_data = {k: v for k, v in res.items() if k not in ['question', 'model_id']}
    answer_data = remove_contexts(answer_data)
    structured_results[question][model_id] = answer_data

structured_results = dict(structured_results)

with open("FIVE_results_by_question.json", "w", encoding="utf-8") as f:
    json.dump(structured_results, f, ensure_ascii=False, indent=4)

print("Results saved to FIVE_results_by_question.json")


Results saved to FIVE_results_by_question.json
